## model

### BILSTM

In [70]:
import keras
import pickle
import numpy as np
import pandas as pd
from tflearn.data_utils import pad_sequences
from collections import defaultdict
from sklearn.metrics import f1_score, precision_score, recall_score
from keras.layers import *
from keras.models import Sequential
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
from keras import initializers
import os
import keras
import jieba
from collections import Counter 
jieba.add_word('花呗')
jieba.add_word('借呗')
jieba.add_word('收钱码')
jieba.add_word('收款码')

from keras.optimizers import Adadelta
import gc

config = tf.ConfigProto()  
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess = tf.Session(config=config)

KTF.set_session(sess)

EMBEDDING_DIM = 300
VOCAB_LENGTH = 3000

from tensorflow.python.client import device_lib


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)



embedding_matrix_char = np.load('../data/aux/vec_char.npy')

embedding_matrix_word = np.load('../data/aux/vec_word.npy')

In [92]:
train_data_char = pd.read_csv('../dfc_split.csv', header=None)
train_data_word = pd.read_csv('../dfw_split.csv', header=None)
print train_data_char.head()

       0                                                  1                                                  2  3
0   1002       65 91 10 26 27 6 2 52 15 4 97 21 23 38 31 35                           65 91 52 70 79 31 35 6 2  1
1  39267                 3 2 83 48 242 58 35 229 15 4 65 91                      15 4 105 83 242 58 35 229 3 2  1
2   3732              12 263 443 134 6 2 184 162 310 259 10                   6 2 184 162 310 259 20 513 175 9  0
3  17937  24 17 4 40 59 38 7 9 3 2 5 12 18 383 340 51 11...  7 9 3 2 116 103 9 56 75 54 73 98 12 11 10 7 59...  0
4   6929                               26 27 6 2 6 12 10 33                         15 4 7 9 26 27 6 2 6 12 10  1


In [10]:
jieba.lcut('花呗分期查询'.decode('UTF-8'))

[u'\u82b1\u5457', u'\u5206\u671f', u'\u67e5\u8be2']

In [86]:
### 分词、分字
PAD_ID = 0
UNK_ID=1

def translate(text, translation):
    for token, replacement in translation.items():
        text = text.replace(token, ' ' + replacement + ' ')
    text = text.replace('  ', ' ')
    return text
def token_string_as_list(string, token_string='char'):
    string = string.decode('UTF-8')
    translation = {
        '***':'*',
        u'花被':u'花呗'
    }
    translate(string, translation)
    length = len(string)
    if token_string == 'char':
        listt = [string[i] for i in range(length)]
    elif token_string == 'word':
        listt = jieba.lcut(string)
    elif token_string == 'pinyin':
        string = ''.join(jibe.lcut(string))
        listt = ''.join(lazy_pinyin(string)).split()
    listt = [item for item in listt if item.strip()]
    return listt
def create_voca(train_data, token_string='char', voca_size=5000):
    size = train_data.shape[0]
    input_count = Counter()
    for i in range(size):
        token_list = token_string_as_list(train_data.iloc[i, 1], token_string=token_string)
        input_count.update(token_list)
        token_list = token_string_as_list(train_data.iloc[i, 2], token_string=token_string)
        input_count.update(token_list)
    vocab_worddict = {}
    vocab_indexdict = {}
    vocab_list = input_count.most_common(voca_size)
    for i, tuplee in enumerate(vocab_list):
        word,_ = tuplee
        vocab_worddict[i+2] = word
        vocab_indexdict[word] = i+2
    return vocab_worddict, vocab_indexdict
def parse_train_data(train_data, vocab_indexdict, token_string):
    size = train_data.shape[0]
    input_count = Counter()
    parse_data = train_data.copy()
    temp_data = train_data.copy()
    for i in range(size):
        token_list1 = token_string_as_list(train_data.iloc[i, 1], token_string=token_string)
        token_list2 = token_string_as_list(train_data.iloc[i, 2], token_string=token_string)
        temp_data.iloc[i, 1] = ' '.join(token_list1)
        temp_data.iloc[i, 2] = ' '.join(token_list2)
        parse_data.iloc[i, 1] = ' '.join([str(vocab_indexdict[item]) for item in token_list1 if item in vocab_indexdict])
        parse_data.iloc[i, 2] = ' '.join([str(vocab_indexdict[item]) for item in token_list2 if item in vocab_indexdict])
    return parse_data

def parse_split(train_data):
    vocab_worddict_word, vocab_indexdict_word = create_voca(train_data, token_string='char')
    parse_data = parse_train_data(train_data, vocab_indexdict_word,token_string='char')
    return parse_data

train_data_char_df = parse_split(train_data_char)

def parse_split2(train_data):
    vocab_worddict_word, vocab_indexdict_word = create_voca(train_data, token_string='word')
    parse_data = parse_train_data(train_data, vocab_indexdict_word,token_string='word')
    return parse_data
train_data_word_df = parse_split2(train_data_char)

train_data_char_df.to_csv('../dfc_split.csv', index=False, header=False)
train_data_word_df.to_csv('../dfw_split.csv', index=False, header=False)

In [87]:
train_data_word_df.head()

,0,1,2,3
0,1002,48 7 13 3 45 10 136 14 29 19,48 45 49 19 3,1
1,39267,2 109 217 417 10 48,10 77 217 417 2,1
2,3732,25 490 75 3 352 247 7,3 352 247 18 693 5,0
3,17937,15 32 39 4 5 2 11 21 430 28 115 26 43 14 88,4 5 2 76 5 1504 56 119 7 4 39 52 106 162 67 31...,0
4,6929,13 3 54 31 23,10 4 5 13 3 54 31,1


In [122]:

VOC_SIZE_CHAR = 2087
VOC_SIZE_WORD = 5002
from keras.layers import *
from keras.activations import softmax
from keras.models import Model

def unchanged_shape(input_shape):
    return input_shape

def time_distributed(x, layers):
    for l in layers:
        x = TimeDistributed(l)(x)
    return x

def align(input_1, input_2):
    attention = Dot(axes=-1)([input_1, input_2])
    w_att_1 = Lambda(lambda x: softmax(x, axis=1),
                     output_shape=unchanged_shape)(attention)
    w_att_2 = Permute((2,1))(Lambda(lambda x: softmax(x, axis=2),
                             output_shape=unchanged_shape)(attention))
    in1_aligned = Dot(axes=1)([w_att_1, input_1])
    in2_aligned = Dot(axes=1)([w_att_2, input_2])
    return in1_aligned, in2_aligned

def aggregate_both(x1, x2, num_class, dense_dim=100, dropout_rate=0.2, activation="relu"):#
    feat1 = concatenate(list(map(lambda l: l(x1), [GlobalAvgPool1D(), GlobalMaxPool1D()])))
    feat2 = concatenate(list(map(lambda l: l(x2), [GlobalAvgPool1D(), GlobalMaxPool1D()])))
    feat3 = add([feat1, feat2])
    feat4 = subtract([feat1, feat2])
    feat5 = Lambda(lambda x: abs(x[0]-x[1]))([feat1, feat2])
    x = Concatenate()([feat1, feat2]) #, feat4, feat3
    x = BatchNormalization()(x)
    x = Dense(dense_dim, activation=activation)(x)
    x = Dropout(dropout_rate)(x)
    x = BatchNormalization()(x)
    x = Dense(dense_dim, activation=activation)(x)
    x = Dropout(dropout_rate)(x)
    return x

def lstm_project_atten(q1_embed, q2_embed, projection_dim=100, projection_hidden=200, projection_dropout=0.2,
                           compare_dim=500, compare_dropout=0.2,
                           dense_dim=300, dropout_rate=0.2,
                           lr=1e-3, activation='relu'):
    
    # CuDNN
    lstm0 = CuDNNLSTM(model_params['num_lstm'],
                 return_sequences = True)
    lstm1 = Bidirectional(CuDNNLSTM(model_params['num_lstm'],
                 return_sequences = True))
    lstm2 = CuDNNLSTM(model_params['num_lstm'])
    
    att1 = Lambda(lambda x: K.max(x,axis = 1))
    q1_embed = lstm1(q1_embed)
    q2_embed = lstm1(q2_embed)
    
    # Projection
    projection_layers = []
#     if projection_hidden > 0:
#         projection_layers.extend([
#                 Dense(projection_hidden, activation=activation),
#                 Dropout(rate=projection_dropout),
#             ])
#     projection_layers.extend([
#             Dense(projection_dim, activation=None),
#             Dropout(rate=projection_dropout),
#         ])
    q1_encoded = time_distributed(q1_embed, projection_layers)
    q2_encoded = time_distributed(q2_embed, projection_layers)
    
    # Attention
    q1_aligned, q2_aligned = align(q1_encoded, q2_encoded)  
    
    # Compare
    q1_combined = concatenate([q1_encoded, q2_aligned])
    q2_combined = concatenate([q2_encoded, q1_aligned]) 
#     compare_layers = [
#         Dense(compare_dim, activation=None),
#         Dropout(compare_dropout),
#         Dense(compare_dim, activation=activation),
#         Dropout(compare_dropout),
#     ]
    compare_layers = []
    q1_compare = time_distributed(q1_combined, compare_layers)
    q2_compare = time_distributed(q2_combined, compare_layers)
    return q1_compare, q2_compare

def build_model_combine(params, num_class=1,projection_dim=300, projection_hidden=200, projection_dropout=0.2,
                           compare_dim=500, compare_dropout=0.2,
                           dense_dim=300, dropout_rate=0.2,
                           lr=1e-3, activation='relu'):
    q1 = Input(name='q1',shape=(MAX_SEQUENCE_LENGTH_CHAR,), dtype='int32')
    q2 = Input(name='q2',shape=(MAX_SEQUENCE_LENGTH_CHAR,), dtype='int32')
    
    q1_com = Input(name='q1_com',shape=(MAX_SEQUENCE_LENGTH_CHAR, 1), dtype='float32')
    q2_com = Input(name='q2_com',shape=(MAX_SEQUENCE_LENGTH_CHAR, 1), dtype='float32')
    
    encode = Embedding(VOC_SIZE_CHAR, 300, input_length = MAX_SEQUENCE_LENGTH_CHAR,
                       weights = [embedding_matrix_char], trainable=True) # 
    #encode0 = StaticEmbedding(embedding_matrix_char) #, encode0(q1), encode0(q2)
    
    # Word Representation Layer
    q1_embed = concatenate([encode(q1), q1_com])
    q2_embed = concatenate([encode(q2), q2_com])
    
    # Densely-connected Co-attentive networks
    q1_compare0, q2_compare0 = lstm_project_atten(q1_embed, q2_embed)
    
    
    q1_compare0 = concatenate([q1_compare0, q1_embed])
    q2_compare0 = concatenate([q2_compare0, q2_embed])
    
    q1_compare1, q2_compare1 = lstm_project_atten(q1_compare0, q2_compare0)
    
    q1_compare1 = concatenate([q1_compare1, q1_compare0, q1_embed])
    q2_compare1 = concatenate([q2_compare1, q2_compare0, q2_embed])
    
    q1_compare2, q2_compare2 = lstm_project_atten(q1_compare1, q2_compare1)
    
    
    # Aggregate
    x = aggregate_both(q1_compare1, q2_compare1, num_class)#, q10_compare1, q20_compare1
    scores = Dense(num_class, activation='sigmoid')(x)
    
    model = Model(inputs=[q1, q2, q1_com, q2_com], outputs=scores)#
    return model

In [95]:
train_data_char_df = train_data_char
train_data_char_df.head()

,0,1,2,3
0,1002,65 91 10 26 27 6 2 52 15 4 97 21 23 38 31 35,65 91 52 70 79 31 35 6 2,1
1,39267,3 2 83 48 242 58 35 229 15 4 65 91,15 4 105 83 242 58 35 229 3 2,1
2,3732,12 263 443 134 6 2 184 162 310 259 10,6 2 184 162 310 259 20 513 175 9,0
3,17937,24 17 4 40 59 38 7 9 3 2 5 12 18 383 340 51 11...,7 9 3 2 116 103 9 56 75 54 73 98 12 11 10 7 59...,0
4,6929,26 27 6 2 6 12 10 33,15 4 7 9 26 27 6 2 6 12 10,1


In [96]:
#train_data_char_df = train_data_char_df.head(35000)
word_squence_ques1_char = list(train_data_char_df.iloc[:, 1])
word_squence_ques1_char = [[int(im) for im in item.split(' ')] for item in word_squence_ques1_char]
word_squence_ques2_char = list(train_data_char_df.iloc[:, 2])
word_squence_ques2_char = [[int(im) for im in item.split(' ')] for item in word_squence_ques2_char]

# word_squence_ques1_word = list(train_data_word.iloc[:, 1])
# word_squence_ques1_word = [[int(im) for im in item.split(' ')] for item in word_squence_ques1_word]
# word_squence_ques2_word = list(train_data_word.iloc[:, 2])
# word_squence_ques2_word = [[int(im) for im in item.split(' ')] for item in word_squence_ques2_word]


MAX_SEQUENCE_LENGTH_WORD = 30 # char 40 word 30
MAX_SEQUENCE_LENGTH_CHAR = 40 # char 40 word 30
word_squence_ques1_char = pad_sequences(word_squence_ques1_char, maxlen=MAX_SEQUENCE_LENGTH_CHAR)
word_squence_ques2_char = pad_sequences(word_squence_ques2_char, maxlen=MAX_SEQUENCE_LENGTH_CHAR)
def cal_comman_sen(word_squence_ques1_char, word_squence_ques2_char):
    comman_sen1 = []
    comman_sen2 = []
    for ques1, ques2 in zip(word_squence_ques1_char, word_squence_ques2_char):
        comman_sen1.append([[1] if item in ques2 and item!=0 else [0] for item in ques1])
        comman_sen2.append([[1] if item in ques1 and item!=0 else [0] for item in ques2])
    return comman_sen1, comman_sen2
comman_sen1, comman_sen2 = cal_comman_sen(word_squence_ques1_char, word_squence_ques2_char)
# word_squence_ques1_word = pad_sequences(word_squence_ques1_word, maxlen=MAX_SEQUENCE_LENGTH_WORD)
# word_squence_ques2_word = pad_sequences(word_squence_ques2_word, maxlen=MAX_SEQUENCE_LENGTH_WORD)


In [123]:
class Metrics(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.val_f1s = []

    def on_epoch_end(self, epoch, logs={}):
        #import pdb; pdb.set_trace()
        val_predict = (np.asarray(self.model.predict(self.validation_data[:4])))#
        val_label = self.validation_data[-2]
        for weight in range(10, 90, 5):
            val_pred = [1 if item > weight*0.01 else 0 for item in val_predict]
            _val_f1 = f1_score(val_label, val_pred)
            print("epoch end ", weight*0.01, ", f1-score: ", _val_f1, "precision: ", 
                  precision_score(val_label, val_pred, average='binary'), "recall: ", 
                  recall_score(val_label, val_pred, average='binary'))
metrics = Metrics()

from sklearn.model_selection import StratifiedKFold
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
X_train_q1 = word_squence_ques1_char
X_train_q2 = word_squence_ques2_char
X_train_q1_com = np.array(comman_sen1)
X_train_q2_com = np.array(comman_sen2)
y_train = train_data_char_df.iloc[:, 3]

class_weight = {0: 1.,
                1: 3.,}

X_fold_val_q1_out,X_fold_val_q2_out, X_fold_val_magic_out = [0, 0, 0]

def model_train(model_func, X_train_q1, X_train_q2, y_train, model_checkpoint_path):
    NUM_FOLDS = 10
    kfold = StratifiedKFold(
        n_splits=NUM_FOLDS,
        shuffle=True,
        random_state=30
    )
    BATCH_SIZE = 128
    MAX_EPOCHS = 100

    for fold_num, (ix_train, ix_val) in enumerate(kfold.split(X_train_q1, y_train)):

        # Augment the training set by mirroring the pairs.
        #ix_train = ix_val = int(X_train_q1.shape[0]*0.8)
        X_fold_train_q1 = np.vstack([X_train_q1[ix_train], X_train_q2[ix_train]])
        X_fold_train_q2 = np.vstack([X_train_q2[ix_train], X_train_q1[ix_train]])
        X_fold_train_q1_com = np.vstack([X_train_q1_com[ix_train], X_train_q2_com[ix_train]])
        X_fold_train_q2_com = np.vstack([X_train_q2_com[ix_train], X_train_q1_com[ix_train]])
#         X_fold_train_q10 = np.vstack([X_train_q10[ix_train], X_train_q20[ix_train]])
#         X_fold_train_q20 = np.vstack([X_train_q20[ix_train], X_train_q10[ix_train]])
        #X_fold_train_magic = np.vstack([X_train_magic[ix_train], X_train_magic[ix_train]])

        X_fold_val_q1 = np.vstack([X_train_q1[ix_val], X_train_q2[ix_val]])
        X_fold_val_q2 = np.vstack([X_train_q2[ix_val], X_train_q1[ix_val]])
        X_fold_val_q1_com = np.vstack([X_train_q1_com[ix_val], X_train_q2_com[ix_val]])
        X_fold_val_q2_com = np.vstack([X_train_q2_com[ix_val], X_train_q1_com[ix_val]])
#         X_fold_val_q10 = np.vstack([X_train_q10[ix_val], X_train_q20[ix_val]])
#         X_fold_val_q20 = np.vstack([X_train_q20[ix_val], X_train_q10[ix_val]])
        #X_fold_val_magic = np.vstack([X_train_magic[ix_val], X_train_magic[ix_val]])
        
        
        # Ground truth should also be "mirrored".
        y_fold_train = np.concatenate([y_train[ix_train], y_train[ix_train]])
        y_fold_val = np.concatenate([y_train[ix_val], y_train[ix_val]])

        print('Fitting fold')

        # Compile a new model.
        # model1 = create_model_cnn(model_params)
        model = model_func(model_params)

        model.compile(optimizer= keras.optimizers.Adam(lr = 0.0005), loss="binary_crossentropy", 
            metrics=['accuracy'])
        # Train.
        model.fit(
            [X_fold_train_q1, X_fold_train_q2, 
             X_fold_train_q1_com, X_fold_train_q2_com,
            # X_fold_train_q10, X_fold_train_q20, 
#              X_fold_train_char0_len, X_fold_train_char1_len,
#              X_fold_train_word0_len, X_fold_train_word1_len,
             # X_fold_train_magic
            ], y_fold_train,#
            validation_data=([
                X_fold_val_q1, X_fold_val_q2, 
             X_fold_val_q1_com, X_fold_val_q2_com,
                           #   X_fold_val_q10, X_fold_val_q20, 
#                               X_fold_val_char0_len, X_fold_val_char1_len,
#                               X_fold_val_word0_len,X_fold_val_word1_len,
                              # X_fold_val_magic
                             ], y_fold_val),#

            batch_size=BATCH_SIZE,
            epochs=MAX_EPOCHS,
            verbose=1,
            shuffle=True,
            # class_weight=class_weight,

            callbacks=[
                # Stop training when the validation loss stops improving.
                EarlyStopping(
                    monitor='val_loss',
                    min_delta=0.001,
                    patience=20,
                    verbose=1,
                    mode='auto',
                ),
                ReduceLROnPlateau(
                    monitor='val_loss', 
                    factor=0.1, 
                    patience=5, 
                    min_delta=0.0001,
                    cooldown=1, 
                ),
                # Save the weights of the best epoch.
#                 ModelCheckpoint(
#                     os.path.join(model_checkpoint_path + str(fold_num) + 'weights.{epoch:02d}.hdf5'),
#                     monitor='val_loss',
#                     save_best_only=False,
#                     verbose=2,
#                     mode='auto',
#                 ),
                metrics
            ],
        )
        break

model_path = '../data/checkpoints/test/'


model_params = {
    'dense_dropout_rate': 0.3,
    'num_dense': 150,
    'num_lstm': 128,
    'num_filters':32
}

model_train(build_model_combine, X_train_q1, X_train_q2, y_train, model_path)


Fitting fold
Train on 89684 samples, validate on 9968 samples
Epoch 1/100
89684/89684 [==============================] - 71s 787us/step - loss: 0.5832 - acc: 0.6968 - val_loss: 0.5412 - val_acc: 0.7197
('epoch end ', 0.1, ', f1-score: ', 0.9457987414732166, 'precision: ', 1.0, 'recall: ', 0.8971709470304976)
('epoch end ', 0.15, ', f1-score: ', 0.9014162120460683, 'precision: ', 1.0, 'recall: ', 0.8205256821829856)
('epoch end ', 0.2, ', f1-score: ', 0.8544503821180256, 'precision: ', 1.0, 'recall: ', 0.7458868378812199)
('epoch end ', 0.25, ', f1-score: ', 0.8008421052631579, 'precision: ', 1.0, 'recall: ', 0.6678370786516854)
('epoch end ', 0.3, ', f1-score: ', 0.7532207629768606, 'precision: ', 1.0, 'recall: ', 0.6041332263242376)
('epoch end ', 0.35000000000000003, ', f1-score: ', 0.7021678276153895, 'precision: ', 1.0, 'recall: ', 0.5410313001605136)
('epoch end ', 0.4, ', f1-score: ', 0.6468472137378674, 'precision: ', 1.0, 'recall: ', 0.47802969502407705)
('epoch end ', 0.45, ',

('epoch end ', 0.7000000000000001, ', f1-score: ', 0.37562128249001875, 'precision: ', 1.0, 'recall: ', 0.23123996789727128)
('epoch end ', 0.75, ', f1-score: ', 0.328498365054079, 'precision: ', 1.0, 'recall: ', 0.19652889245585875)
('epoch end ', 0.8, ', f1-score: ', 0.26568073075250104, 'precision: ', 1.0, 'recall: ', 0.15319020866773675)
('epoch end ', 0.85, ', f1-score: ', 0.18763636363636363, 'precision: ', 1.0, 'recall: ', 0.10353130016051364)
Epoch 6/100
89684/89684 [==============================] - 56s 627us/step - loss: 0.3523 - acc: 0.8434 - val_loss: 0.5812 - val_acc: 0.7470
('epoch end ', 0.1, ', f1-score: ', 0.7455323433173924, 'precision: ', 1.0, 'recall: ', 0.5943017656500803)
('epoch end ', 0.15, ', f1-score: ', 0.7028433860368274, 'precision: ', 1.0, 'recall: ', 0.5418338683788122)
('epoch end ', 0.2, ', f1-score: ', 0.6663990902401499, 'precision: ', 1.0, 'recall: ', 0.49969903691813805)
('epoch end ', 0.25, ', f1-score: ', 0.6324598710385513, 'precision: ', 1.0, 'r

89684/89684 [==============================] - 56s 629us/step - loss: 0.1133 - acc: 0.9580 - val_loss: 0.9219 - val_acc: 0.7421
('epoch end ', 0.1, ', f1-score: ', 0.6696029362696029, 'precision: ', 1.0, 'recall: ', 0.5033105939004815)
('epoch end ', 0.15, ', f1-score: ', 0.6495054870613738, 'precision: ', 1.0, 'recall: ', 0.4809390048154093)
('epoch end ', 0.2, ', f1-score: ', 0.630957286087076, 'precision: ', 1.0, 'recall: ', 0.46087479935794545)
('epoch end ', 0.25, ', f1-score: ', 0.6149784632485757, 'precision: ', 1.0, 'recall: ', 0.44402086677367575)
('epoch end ', 0.3, ', f1-score: ', 0.601080625920988, 'precision: ', 1.0, 'recall: ', 0.4296749598715891)
('epoch end ', 0.35000000000000003, ', f1-score: ', 0.5882019686991006, 'precision: ', 1.0, 'recall: ', 0.41663322632423755)
('epoch end ', 0.4, ', f1-score: ', 0.5746764853077858, 'precision: ', 1.0, 'recall: ', 0.40319020866773675)
('epoch end ', 0.45, ', f1-score: ', 0.5605776173285199, 'precision: ', 1.0, 'recall: ', 0.38944

('epoch end ', 0.7000000000000001, ', f1-score: ', 0.4943735367419379, 'precision: ', 1.0, 'recall: ', 0.32835072231139645)
('epoch end ', 0.75, ', f1-score: ', 0.4811823861039159, 'precision: ', 1.0, 'recall: ', 0.31681380417335475)
('epoch end ', 0.8, ', f1-score: ', 0.4663435648896069, 'precision: ', 1.0, 'recall: ', 0.30407303370786515)
('epoch end ', 0.85, ', f1-score: ', 0.4456572586932793, 'precision: ', 1.0, 'recall: ', 0.2867174959871589)
Epoch 15/100
89684/89684 [==============================] - 56s 627us/step - loss: 0.0521 - acc: 0.9824 - val_loss: 1.2434 - val_acc: 0.7386
('epoch end ', 0.1, ', f1-score: ', 0.6303929651003022, 'precision: ', 1.0, 'recall: ', 0.4602728731942215)
('epoch end ', 0.15, ', f1-score: ', 0.6129548458916023, 'precision: ', 1.0, 'recall: ', 0.44191412520064205)
('epoch end ', 0.2, ', f1-score: ', 0.599213041034289, 'precision: ', 1.0, 'recall: ', 0.42776886035313)
('epoch end ', 0.25, ', f1-score: ', 0.5876018420120439, 'precision: ', 1.0, 'recall

89684/89684 [==============================] - 56s 627us/step - loss: 0.0462 - acc: 0.9843 - val_loss: 1.2906 - val_acc: 0.7390
('epoch end ', 0.1, ', f1-score: ', 0.6282254180141746, 'precision: ', 1.0, 'recall: ', 0.45796548956661315)
('epoch end ', 0.15, ', f1-score: ', 0.6125687243371146, 'precision: ', 1.0, 'recall: ', 0.4415128410914928)
('epoch end ', 0.2, ', f1-score: ', 0.5987207422506502, 'precision: ', 1.0, 'recall: ', 0.4272672552166934)
('epoch end ', 0.25, ', f1-score: ', 0.5888015856161959, 'precision: ', 1.0, 'recall: ', 0.4172351524879615)
('epoch end ', 0.3, ', f1-score: ', 0.5787409995009625, 'precision: ', 1.0, 'recall: ', 0.40720304975922955)
('epoch end ', 0.35000000000000003, ', f1-score: ', 0.5689469528389922, 'precision: ', 1.0, 'recall: ', 0.39757223113964685)
('epoch end ', 0.4, ', f1-score: ', 0.5607854461449611, 'precision: ', 1.0, 'recall: ', 0.38964686998394865)
('epoch end ', 0.45, ', f1-score: ', 0.5501090909090909, 'precision: ', 1.0, 'recall: ', 0.379

('epoch end ', 0.7000000000000001, ', f1-score: ', 0.49562330214307276, 'precision: ', 1.0, 'recall: ', 0.329454253611557)
('epoch end ', 0.75, ', f1-score: ', 0.4844153869545386, 'precision: ', 1.0, 'recall: ', 0.3196227929373997)
('epoch end ', 0.8, ', f1-score: ', 0.47116564417177914, 'precision: ', 1.0, 'recall: ', 0.30818619582664525)
('epoch end ', 0.85, ', f1-score: ', 0.45133224578575315, 'precision: ', 1.0, 'recall: ', 0.2914325842696629)
Epoch 00023: early stopping


In [46]:
train_data_char_df.iloc[:, 3].value_counts()

0    27405
1     7595
Name: 3, dtype: int64

0.0